---
layout: code-post
title: Pedicting rooting interests on reddit
description: Going to see if we can predict neutral fan rooting interests from reddit posts.
tags: [neural nets]
---

In this notebook / post I'm going to try to see if we can predict who a
supposedly neutral fan is rooting for in the (as of this writing) about to
end Lakers-Heat NBA finals. To do this, I'm going to attempt to scrape posts
from game threads by Heat and Lakers fans to train an LSTM based model. As long
as I can scrape the flair from from the reddit API, this should be doable. If
not, then I will have to scrape from the team specific subreddits and try that
way.

Outline:
- Accessing Reddit
  - Reddit's API
  - Getting posts
- Making a dataset
  - Encoding posts
  - Saving the data
- Training a model
- Test set predictions

## Accessing reddit

If you don't know what [reddit](https://reddit.com/) is, it's a website that is
organized into communities called _subreddits_. The site has _users_ which belong to
multiple subreddits. Each subreddit contains a sequence of _submissions_ (also called
_posts_) which can be links
to other sites, images, or text. Each post contains _comments_ which are text only
an are made by the users. The _comment section_ is organized as a forest of trees
wich top level comments and then comments nested below each top level comment. Each
user can have a _flair_ which varies with the subreddit they are posting in. The
flair and username are posted along with each comment. The flair will contains a
small image as well as text. Not every user has a flair associated to it for every
subreddit to which they belong. It is not always required to be a member of the
subreddit in order to post, although this varies by community.

We will be using the NBA subreddit which as of this writing has ~3.5 million users.
Users in this subreddit have flairs which denote which team the user is a supporter
of. Mine is for the Cleveland Cavaliers.

In order to access reddit, we need to have a reddit account and then create an application
with that account as a developer of the application. Go to [the apps page](https://www.reddit.com/prefs/apps)
to create an application. (Please read the terms and conditions!)
You need to store the name of the app as the `user_agent`, the
`client_id` is the 14 character string that is below the app name once it's created, 
you'll have a 27 character secret secret that is generated and is the `client_secret`, 
and you'll need your `username`. I have put these credentials in an encrypted YAML file
that I created using ansible-vault.

We will be using [PRAW](https://github.com/praw-dev/praw), the Python Reddit API
Wrapper.

In [9]:
import praw
import yaml
from ansible_vault import Vault
from getpass import getpass

In [10]:
vault = Vault(getpass())
with open('redditcreds.yml', 'r') as f:
    reddit_creds = vault.load(f.read())

reddit = praw.Reddit(username=reddit_creds['username'],
                     user_agent=reddit_creds['user_agent'],
                     client_id=reddit_creds['client_id'],
                     client_secret=reddit_creds['client_secret'])

 ····················


Now we get into the [NBA subreddit](https://www.reddit.com/r/nba) and search for posts which are _game threads_
to which users will post while a game is ongoing. These are posted automatically and so they have a 
predictible title format which makes searching for them and then filtering the received submisisons
by title easier.

In [76]:
subreddit = reddit.subreddit('nba')

submissions = subreddit.search(query='title:"GAME THREAD"',
                               time_filter="month")
game_threads = [
    s for s in submissions 
    if
        s.title[:11] == 'GAME THREAD' 
        and ("Lakers" in s.title or "Heat" in s.title)
]

In [77]:
for t in game_threads:
    print(t.title)

GAME THREAD: Miami Heat (44-29) @ Los Angeles Lakers (52-19) - (October 09, 2020)
GAME THREAD: Miami Heat (44-29) @ Los Angeles Lakers (52-19) - (September 30, 2020)
GAME THREAD: Los Angeles Lakers (52-19) @ Miami Heat (44-29) - (October 06, 2020)
GAME THREAD: Los Angeles Lakers (52-19) @ Miami Heat (44-29) - (October 04, 2020)
GAME THREAD: Miami Heat (44-29) @ Los Angeles Lakers (52-19) - (October 02, 2020)
GAME THREAD: Denver Nuggets (46-27) @ Los Angeles Lakers (52-19) - (September 26, 2020)
GAME THREAD: Los Angeles Lakers (52-19) @ Denver Nuggets (46-27) - (September 24, 2020)
GAME THREAD: Los Angeles Lakers (52-19) @ Denver Nuggets (46-27) - (September 22, 2020)
GAME THREAD: Denver Nuggets (46-27) @ Los Angeles Lakers (52-19) - (September 18, 2020)
GAME THREAD: Boston Celtics (48-24) @ Miami Heat (44-29) - (September 27, 2020)
GAME THREAD: Denver Nuggets (46-27) @ Los Angeles Lakers (52-19) - (September 20, 2020)
GAME THREAD: Miami Heat (44-29) @ Boston Celtics (48-24) - (Septembe

The comments for a submission are contained in PRAW `CommentForest`. If we do not
care about the structure, we can flatten this to a list. Note that the list
will contain both `Comment` objects as well as `MoreComments` objectcs. It is possible to replace
the `MoreComments` objects using the `replace_more` function, but each replacement
requires calling the reddit API. By default, 32 of the `MoreComments` objects will
be replaced, which I will keep but I will also limit to those which contain at least
5 more comments in them. Doing this removes all `MoreComments` instances from
the list of comments.

In [93]:
game_thread = game_threads[0]
print(game_thread.title)
print("number of comments:", game_thread.num_comments)
game_thread.comments.replace_more(limit=32, threshold=5)
comments = game_thread.comments.list()

GAME THREAD: Miami Heat (44-29) @ Los Angeles Lakers (52-19) - (October 09, 2020)
number of comments: 28455


And here is a random comment:

In [168]:
c = comments[10]
print('comment author:   ', c.author.name)
print('comment score:    ', c.score, '({}-{})'.format(c.ups, c.downs))
print('author flair text:', c.author_flair_text)
print('body:\n\n', c.body)

comment author:    StoneColdAM
comment score:     28 (28-0)
author flair text: Lakers
body:

 Dwight Howard strategy: get Jimmy Butler ejected.


For the training set, the goal will be to find authors which are flaired as Lakers or Heat or have
flair text that contains LAL or MIA.

In [178]:
lakers_comments = [
    c for c in comments
    if
        c.author_flair_text is not None
        and (
            c.author_flair_text == 'Lakers'
            or '[LAL]' in c.author_flair_text
        )
]
print('number of comments by Lakers users:', len(lakers_comments))

heat_comments = [
    c for c in comments
    if
        c.author_flair_text is not None
        and (
            c.author_flair_text == 'Heat'
            or '[MIA]' in c.author_flair_text
        )
]
print('number of comments by Heat users  :', len(heat_comments))

number of comments by Lakers users: 1042
number of comments by Heat users  : 275


Now that we have an idea of how to get comments for a given fanbase during game threads,
let's try to build up a collection of comments and then clean up the body of those
comments using some common NLP techniques. For example, we will require comments to 
have some minimum number of words in them, we will then remove stopwords and spaces
and other weird text as best we can. Then we will vectorize each word. To do all this
we will rely on `spaCy`, which I wrote about in [this post](https://kevinnowland.com/code/2020/10/04/nlp-intro.html).

In [ ]:
import spacey
